Solving a Finite Markov Decision Processes with Q-Learning:
game with map like environment, treasure, trees, monsters, obstacles, rewards

https://medium.com/@curiousily/solving-an-mdp-with-q-learning-from-scratch-deep-reinforcement-learning-for-hackers-part-1-45d1d360c120

https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

In [1]:
import numpy as np
import random

Building the environment

In [2]:
MONSTER = "m"
PLAYER = "p"
REWARD = "r"
EMPTY = "*"
TREE = "t"
END = "e"

grid = [
            [PLAYER, EMPTY, MONSTER, MONSTER],
            [EMPTY, EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY, EMPTY],
            [EMPTY, TREE, EMPTY, EMPTY],
            [REWARD, TREE, EMPTY, EMPTY],
            [EMPTY, TREE, EMPTY, EMPTY],
            [EMPTY, TREE, EMPTY, MONSTER],
            [EMPTY, TREE, EMPTY, EMPTY],
            [MONSTER, TREE, EMPTY, END]
        ]



for row in grid:
    print(' '.join(row))

p * m m
* * * *
* * * *
* * * *
* t * *
r t * *
* t * *
* t * m
* t * *
m t * e


Wrap our environment state in a class that holds the current grid and player position

In [3]:
class State:
    
    def __init__(self, grid, player_pos):
        self.grid = grid
        self.player_pos = player_pos
        
    def __eq__(self, other):
        return isinstance(other, State) and self.grid == other.grid and self.player_pos == other.player_pos
    
    def __hash__(self):
        return hash(str(self.grid) + str(self.player_pos))
    
    def __str__(self):
        return f"State(grid={self.grid}, player_pos={self.player_pos})"

Actions and init state

In [4]:
UP = 0
DOWN = 1
LEFT = 2
RIGHT = 3

ACTIONS = [UP, DOWN, LEFT, RIGHT]

start_state = State(grid=grid, player_pos=[0, 0])

Function that takes the current state with an action and returns new state, reward and whether or not the episode has completed


In [5]:
from copy import deepcopy

def act(state, action):
    
    def new_player_pos(state, action):
        p = deepcopy(state.player_pos)
        if action == UP:
            p[0] = max(0, p[0] - 1)
        elif action == DOWN:
            p[0] = min(len(state.grid) - 1, p[0] + 1)
        elif action == LEFT:
            p[1] = max(0, p[1] - 1)
        elif action == RIGHT:
            p[1] = min(len(state.grid[0]) - 1, p[1] + 1)
        else:
            raise ValueError(f"Unknown action {action}")
        return p
            
    p = new_player_pos(state, action)
    grid_item = state.grid[p[0]][p[1]]
    
    new_grid = deepcopy(state.grid)
    
    if grid_item == MONSTER:
        reward = -1000
        is_done = True
        new_grid[p[0]][p[1]] += PLAYER
    elif grid_item == REWARD:
        reward = 1000
        is_done = False
        old = state.player_pos
        new_grid[old[0]][old[1]] = EMPTY
        new_grid[p[0]][p[1]] = PLAYER
    elif grid_item == EMPTY:
        reward = -1
        is_done = False
        old = state.player_pos
        new_grid[old[0]][old[1]] = EMPTY
        new_grid[p[0]][p[1]] = PLAYER
    elif grid_item == END:
        reward = 5000
        is_done = True
        new_grid[p[0]][p[1]] += PLAYER
    elif grid_item == TREE:
        reward = -20
        is_done = False
    elif grid_item == PLAYER:
        reward = -1
        is_done = False
    else:
        raise ValueError(f"Unknown grid item {grid_item}")
    
    return State(grid=new_grid, player_pos=p), reward, is_done

Learning

In [6]:
import numpy as np
import random

random.seed(42) 

N_STATES = len(grid[0]) * len(grid)
N_EPISODES = 1000

MAX_EPISODE_STEPS = 100

MIN_ALPHA = 0.02

alphas = np.linspace(1.0, MIN_ALPHA, N_EPISODES)
gamma = 1.0
eps = 0.2

q_table = dict()

A helper function q that gives the Q value for a state-action pair or for all actions, given a state

In [7]:
def q(state, action=None):
    
    if state not in q_table:
        q_table[state] = np.zeros(len(ACTIONS))
        
    if action is None:
        return q_table[state]
    
    return q_table[state][action]

Act with random action with some small probability or the best action seen so far (using our q_table)

In [8]:
def choose_action(state):
    if random.uniform(0, 1) < eps:
        return random.choice(ACTIONS) 
    else:
        return np.argmax(q(state))

Training agent using the Q-learning algorithm

In [9]:
for e in range(N_EPISODES):
    
    state = start_state
    total_reward = 0
    alpha = alphas[e]
    
    for _ in range(MAX_EPISODE_STEPS):
        action = choose_action(state)
        next_state, reward, done = act(state, action)
        total_reward += reward
        
        q(state)[action] = q(state, action) + \
                alpha * (reward + gamma *  np.max(q(next_state)) - q(state, action))
        state = next_state
        if done:
            break
    print(f"Episode {e + 1}: total reward -> {total_reward}")

Episode 1: total reward -> -1006
Episode 2: total reward -> -1088
Episode 3: total reward -> -1074
Episode 4: total reward -> -1002
Episode 5: total reward -> -1005
Episode 6: total reward -> -1065
Episode 7: total reward -> -1021
Episode 8: total reward -> -120
Episode 9: total reward -> -55
Episode 10: total reward -> -129
Episode 11: total reward -> -1026
Episode 12: total reward -> -12
Episode 13: total reward -> -10
Episode 14: total reward -> -1098
Episode 15: total reward -> -136
Episode 16: total reward -> -319
Episode 17: total reward -> -1029
Episode 18: total reward -> -67
Episode 19: total reward -> -139
Episode 20: total reward -> -187
Episode 21: total reward -> 806
Episode 22: total reward -> -143
Episode 23: total reward -> -49
Episode 24: total reward -> -14
Episode 25: total reward -> -12
Episode 26: total reward -> -1064
Episode 27: total reward -> -126
Episode 28: total reward -> -219
Episode 29: total reward -> -100
Episode 30: total reward -> -25
Episode 31: total

Episode 247: total reward -> 5959
Episode 248: total reward -> 5958
Episode 249: total reward -> 5985
Episode 250: total reward -> 5984
Episode 251: total reward -> 5982
Episode 252: total reward -> 5982
Episode 253: total reward -> 5963
Episode 254: total reward -> -36
Episode 255: total reward -> 5984
Episode 256: total reward -> 5983
Episode 257: total reward -> 768
Episode 258: total reward -> 5980
Episode 259: total reward -> -124
Episode 260: total reward -> 5984
Episode 261: total reward -> 5965
Episode 262: total reward -> -14
Episode 263: total reward -> 5984
Episode 264: total reward -> 5934
Episode 265: total reward -> 5963
Episode 266: total reward -> -61
Episode 267: total reward -> 5940
Episode 268: total reward -> 5942
Episode 269: total reward -> 5960
Episode 270: total reward -> 5898
Episode 271: total reward -> 5980
Episode 272: total reward -> -12
Episode 273: total reward -> 5982
Episode 274: total reward -> 5986
Episode 275: total reward -> 5983
Episode 276: total 

Episode 512: total reward -> 5977
Episode 513: total reward -> -138
Episode 514: total reward -> -44
Episode 515: total reward -> 5982
Episode 516: total reward -> 5963
Episode 517: total reward -> 5982
Episode 518: total reward -> 5980
Episode 519: total reward -> -34
Episode 520: total reward -> 5986
Episode 521: total reward -> 5984
Episode 522: total reward -> 5974
Episode 523: total reward -> 5959
Episode 524: total reward -> 5979
Episode 525: total reward -> 5959
Episode 526: total reward -> 5980
Episode 527: total reward -> 5962
Episode 528: total reward -> 5955
Episode 529: total reward -> 5953
Episode 530: total reward -> 5962
Episode 531: total reward -> -181
Episode 532: total reward -> 5977
Episode 533: total reward -> -86
Episode 534: total reward -> 5942
Episode 535: total reward -> 5982
Episode 536: total reward -> 5986
Episode 537: total reward -> -36
Episode 538: total reward -> -16
Episode 539: total reward -> -17
Episode 540: total reward -> 5980
Episode 541: total r

Episode 766: total reward -> 5981
Episode 767: total reward -> -18
Episode 768: total reward -> 5980
Episode 769: total reward -> 5979
Episode 770: total reward -> 5964
Episode 771: total reward -> 5959
Episode 772: total reward -> -1011
Episode 773: total reward -> -20
Episode 774: total reward -> 5938
Episode 775: total reward -> 5975
Episode 776: total reward -> 5944
Episode 777: total reward -> 5965
Episode 778: total reward -> 5986
Episode 779: total reward -> 5964
Episode 780: total reward -> 5982
Episode 781: total reward -> 5983
Episode 782: total reward -> 5963
Episode 783: total reward -> 5975
Episode 784: total reward -> -94
Episode 785: total reward -> -63
Episode 786: total reward -> 5965
Episode 787: total reward -> 5979
Episode 788: total reward -> 5936
Episode 789: total reward -> -33
Episode 790: total reward -> 5984
Episode 791: total reward -> -102
Episode 792: total reward -> 5980
Episode 793: total reward -> 5961
Episode 794: total reward -> -35
Episode 795: total 